# Fluent Python: Chapter 7.
## Function Decorators and Closures

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [3]:
@deco
def target():
    print('running target()')

In [4]:
target()

running inner()


In [5]:
target

<function __main__.deco.<locals>.inner>

First crucial fact of decorators is they have power to replace decorated function with a different one.
Second crucial fact is they are executed immediately when the module is loaded

### registration.py Example 7-2

In [6]:
registry = []

In [7]:
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

In [8]:
@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(<function f1 at 0x10386af28>)
running register(<function f2 at 0x10387b158>)


In [9]:
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

In [10]:
main()

running main()
registry -> [<function f1 at 0x10386af28>, <function f2 at 0x10387b158>]
running f1()
running f2()
running f3()


f1 and f2 decorated functions are run right away at 'import time

#### Variable scope rules

In [11]:
def f1(a):
    print(a)
    print(b)

In [12]:
f1(3)

3


NameError: name 'b' is not defined

In [13]:
b = 6
f1(3)

3
6


In [14]:
# example 2
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [15]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

When python compiles function, it knows that a local var b exists. Because it hasn't been asigned yet, b is unbound and throws an UnboundLocalError. If we want the function to use the global b definition, include global b

In [ ]:
b = 6
def f3(a):
    global b
    print(a)
    pr